In [1]:
from time import time

import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
import pandas as pd
import torch

from model_svhn import ConvNetBinary, ConvNetClassic
from cifar10_tools import train, test
from datasets import SVHN


This message is expected during the build process. If it appears later on, try installing the package again.


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 256
epochs = 10
lr = 0.0001

dataset_class = SVHN


In [3]:
optimizers = {
    "Adam": optim.Adam,
    "AdaMax": optim.Adamax,
    "AdaDelta": optim.Adadelta
}

models = {
    "Classic_conv": ConvNetClassic,
    "Binary_conv": ConvNetBinary
}


In [4]:
train_kwargs = {"batch_size": batch_size}
test_kwargs = {"batch_size": batch_size}

if device == "cuda":
    cuda_kwargs = {
        "num_workers": 1,
        "pin_memory": True,
        "shuffle": True
        }

    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

train_dataset, test_dataset = dataset_class.get_train_and_test(
    f"./{dataset_class.name}",
    download=True
    )

train_loader = torch.utils.data.DataLoader(train_dataset, **train_kwargs)
test_loader = torch.utils.data.DataLoader(test_dataset, **test_kwargs)


Using downloaded and verified file: svhn/test_32x32.mat
Using downloaded and verified file: svhn/test_32x32.mat


In [5]:
results = []

i = 1
nubmer_of_combinations = len(models) * len(optimizers)
def estimate_time_left(time,number_of_calculation,all_calcualtions):
    seconds = time/number_of_calculation*(all_calcualtions-number_of_calculation)
    return f" {int(seconds/3600)}h {int((seconds%3600)/60)}min {int(seconds%60)}s   "

all_start_train_time = time()

for model_name, model_class in models.items():
    for optimizer_name, optimizer in optimizers.items():
        print(f"Model {model_name}, optimizer {optimizer_name} (combination {i} out of {nubmer_of_combinations})")
        criterion = nn.CrossEntropyLoss()
        model_inctance = model_class().to(device)
        optimizer = optimizer(model_inctance.parameters(), lr=lr)
        for epoch in range(1, epochs+1):
            print(f"Start training epoch {epoch} of {epochs}")
            start_train_time = time()
            train_loss, train_acc = train(
                model_inctance, optimizer, criterion, train_loader, device, epoch
            )
            train_time = time() - start_train_time
            print(f"Finished training of epoch {epoch} in {train_time}, staring testing")

            start_test_time = time()
            test_loss, test_acc = test(
                model_inctance, criterion, test_loader, device
                )
            test_time = time() - start_train_time

            print(f"epoch {epoch}tested. test_acc: {test_acc} test_loss: {test_loss}, epoch train time: {train_time}")
            print(f"estimated time left: {estimate_time_left(time() - all_start_train_time,(i-1)*epochs+epoch,epochs*nubmer_of_combinations)} seconds")

            results.append({
                "model_name": model_name,
                "optimizer_name": optimizer_name,
                "epoch": epoch,
                "train_loss": train_loss,
                "train_acc": train_acc,
                "test_loss": test_loss,
                "test_acc": test_acc,
                "epoch_train_time": train_time,
                "epoch_test_time": test_time
            })
        i = i+1

df_results = pd.DataFrame(results)
df_results.to_csv(f"{dataset_class.name}_results.csv", index=False)


Model Classic_conv, optimizer Adam (combination 1 out of 6)
Start training epoch 1 of 10
Finished training of epoch 1 in 15.734275102615356, staring testing
epoch 1tested. test_acc: 89.4860172095882 test_loss: 0.001435698979701927, epoch train time: 15.734275102615356
estimated time left:  0h 21min 48s    seconds
Start training epoch 2 of 10
Finished training of epoch 2 in 15.55266261100769, staring testing
epoch 2tested. test_acc: 93.55024585125999 test_loss: 0.0009482818825677559, epoch train time: 15.55266261100769
estimated time left:  0h 21min 14s    seconds
Start training epoch 3 of 10
Finished training of epoch 3 in 15.591607332229614, staring testing
epoch 3tested. test_acc: 94.83328211432084 test_loss: 0.0007571456009395004, epoch train time: 15.591607332229614
estimated time left:  0h 20min 50s    seconds
Start training epoch 4 of 10
Finished training of epoch 4 in 15.575006484985352, staring testing
epoch 4tested. test_acc: 96.75015365703749 test_loss: 0.0005186077382772721,